Vocabulary size = 30K

In [1]:
from BERTQG.token_generation import load_model, generate_token
from Regularization_module import Regularizer_Discriminator

class QuestionGeneration():
    def __init__(self, bert_model, lm_gq, uqa_qg, regularization):
        self.lm_qg, _, _ = load_model(bert_model, lm_qg)
        self.uqa_qg, self.tokenizer, device = load_model(bert_model, uqa_qg)
        self.regularization = Regularizer_Discriminator(regularization)
        
    def generate_question(self, context: str, ans: str, ans_start: int, list_question_tokens: list) -> str:
        '''
        Input:
            - context
            - ans
            - ans_start
            - list_question_tokens: the history of generated question tokens.
            The form of list_question_tokens is a list of (q_token, class), where class is 0 for lm
            token and 1 for uqa token
        Output:
            - The next generated question token
        '''
        # contains the tokens of the generated question
        list_question_tokens = []
        # contains the classes of each token of the gen. question. Same len as list_question_tokens
        # 0 = LM, 1 = UQA
        list_token_classes = []
        qi = None
        
        # generation finished when [SEP] is created
        while not self.finished_generation(qi):
            question_text = " ".jon(list_question_tokens)
            
            # Generate the toknes and probs of the ith query token using the lm and uqa models
            lm_qi_token, lm_qi_idx, lm_qi_probs = generate_token(lm_qg, tokenizer, device, context, question_text, ans, ans_start)
            uqa_qi_token, uqa_qi_idx, uqa_qi_probs = generate_token(uqa_qg, tokenizer, device, context, question_text, ans, ans_start)
            
            # Get the final token  prob distribution using regulatization
            jsonobj = self.__convert2squad(context, ans, ans_start, question_text)
            qi_probs = self.regularization.predict_prob(jsonobj) #qi_probs: [batchsize,2] ndarray
            qi_class = np.argmin(qi_probs, axis=1) #qi_class: [batchsize,] ndarray 0:should generate UQA, 1:should generate LM
            qi_class = qi_class[0] #For now, let's assume batch = 1
            # Get the predicted token
            if qi_class == 1: # LM
                qi = lm_qi_token
            else: # UQA
                qi = uqa_qi_token
                
            list_question_tokens.append(qi)
            list_token_classes.append(qi_class)

        return " ".join(list_question_tokens)
            
    def finished_generation(self, question_token):
        return question_token =='[SEP]'
    
    def __convert2squad(self, context: str, answer: str, ans_start: int,  question: str) -> dict:
        '''
        Create a SQuAD instance
        Inputs:
            - context: paragrah
            - answer
            - ans_start
            - question: might not be the full question (we are generating questions token by token)
        Returns:
            - squad instance
        '''
        squad = {'data': [], 'version': '1.0'}
        squad['data'].append({'paragraphs': [{'title': 'title', 
                                              'context': context,
                                              'qas': [{'answers': [{'answer_start': ans_start, 'text': answer}],
                                                       'question': question,
                                                       'id': 0}]}]})
        return squad

04/21/2020 16:03:21 - INFO - transformers.file_utils -   PyTorch version 1.4.0 available.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


04/21/2020 16:03:24 - INFO - transformers.file_utils -   TensorFlow version 2.1.0 available.


In [2]:
bert_model = 'BERTQG/models/bert-base-uncased'
lm_qg = 'BERTQG/models/lm_10k_QG/pytorch_model.bin'
uqa_qg = 'BERTQG/models/uqa_50k_QG/pytorch_model_60000.bin'
regul_model = "./discri_model.bin"# model path here
QG = QuestionGeneration(bert_model, lm_qg, uqa_qg, regul_model)

04/21/2020 16:03:31 - INFO - BERTQG.tokenization -   loading vocabulary file BERTQG/models/bert-base-uncased/vocab.txt
04/21/2020 16:03:36 - INFO - BERTQG.modeling -   loading archive file BERTQG/models/bert-base-uncased
04/21/2020 16:03:36 - INFO - BERTQG.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 3,
  "vocab_size": 30522
}



Load BERTQG/models/lm_10k_QG/pytorch_model.bin


04/21/2020 16:03:40 - INFO - BERTQG.tokenization -   loading vocabulary file BERTQG/models/bert-base-uncased/vocab.txt
04/21/2020 16:03:40 - INFO - BERTQG.modeling -   loading archive file BERTQG/models/bert-base-uncased
04/21/2020 16:03:40 - INFO - BERTQG.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 3,
  "vocab_size": 30522
}



Load BERTQG/models/uqa_50k_QG/pytorch_model_60000.bin


04/21/2020 16:03:44 - INFO - filelock -   Lock 140549952229584 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
04/21/2020 16:03:44 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpvxx3ofi_


04/21/2020 16:03:45 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
04/21/2020 16:03:45 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
04/21/2020 16:03:45 - INFO - filelock -   Lock 140549952229584 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
04/21/2020 16:03:45 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /r

04/21/2020 16:03:46 - INFO - filelock -   Lock 140549951683600 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
04/21/2020 16:03:46 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpyh2oaz_r


04/21/2020 16:03:50 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/21/2020 16:03:50 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/21/2020 16:03:50 - INFO - filelock -   Lock 140549951683600 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
04/21/2020 16:03:50 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/tr

04/21/2020 16:03:51 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
04/21/2020 16:03:51 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
   

FileNotFoundError: [Errno 2] No such file or directory: './discri_model.bin'

In [1]:
import json
with open('../datasets/squad_NER_ans_full_wo_questions_evenly_distributed.json', 'r') as f:
    squad = json.load(f)

# Testing

## Vocabulary

In [9]:
import urllib.request
import numpy as np
word_url = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
response = urllib.request.urlopen(word_url)
long_txt = response.read().decode()
VOCAB_IDX2_STR = np.array(long_txt.splitlines()).reshape(-1,1)

In [10]:
VOCAB_IDX2_STR.shape

(25487, 1)

In [23]:
import torch
import torch.nn as nn

In [24]:
softmax = nn.Softmax(dim=0)

In [29]:
qi_probs = softmax(torch.empty(VOCAB_IDX2_STR.shape[0]).normal_(mean=4,std=0.5))

In [30]:
qi = torch.argmax(qi_probs, dim=0)

In [33]:
VOCAB_IDX2_STR[qi].item()

'Valparaiso'